In [1]:
import tensorflow as tf

print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.7.4


In [2]:
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
print(f"Train Shape(x, y): ({train_x.shape}, {train_y.shape})")

Train Shape(x, y): ((60000, 28, 28), (60000,))


In [3]:
train_x, test_x = train_x / 255., test_x / 255.
train_x, test_x = train_x[..., tf.newaxis].astype("float32"), \
                  test_x[..., tf.newaxis].astype("float32")
print(f"Train Shape(x, y): ({train_x.shape}, {train_y.shape})")                  

Train Shape(x, y): ((60000, 28, 28, 1), (60000,))


In [4]:
SHUFFLE_SIZE = 10000
BATCH_SIZE = 64

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_ds = train_ds.shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(BATCH_SIZE)

2022-12-09 15:22:46.122264: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 15:22:47.571231: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 498 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:5e:00.0, compute capability: 7.5
2022-12-09 15:22:47.572134: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22063 MB memory:  -> device: 1, name: NVIDIA TITAN RTX, pci bus id: 0000:af:00.0, compute capability: 7.5


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation="relu", input_shape=train_x.shape[1:]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(10)
])

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [7]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [8]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

In [9]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    loss = loss_object(labels, predictions)
    test_loss(loss)
    test_accuracy(labels, predictions)

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
    
    for images, labels in test_ds:
        test_step(images, labels)
        
    print(
        f"Epoch: {epoch + 1}, "
        f"Loss: {train_loss.result()}, "
        f"Accuracy: {train_accuracy.result() * 100}, "
        f"Test Loss: {test_loss.result()}, "
        f" Test Accuracy: {test_accuracy.result() * 100}"
    )

2022-12-09 15:38:48.612394: E tensorflow/stream_executor/cuda/cuda_dnn.cc:371] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2022-12-09 15:38:48.612484: E tensorflow/stream_executor/cuda/cuda_dnn.cc:379] Possibly insufficient driver version: 515.65.1
2022-12-09 15:38:48.612537: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1130 : UNKNOWN: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D
 (defined at /home/freeman/anaconda3/envs/tf27p39/lib/python3.9/site-packages/keras/layers/convolutional.py:231)
]] [Op:__inference_train_step_603]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/conv2d/Conv2D:
In[0] images (defined at /tmp/ipykernel_23633/2090703948.py:10)	
In[1] sequential/conv2d/Conv2D/ReadVariableOp:

Operation defined at: (most recent call last)
>>>   File "/home/freeman/anaconda3/envs/tf27p39/lib/python3.9/runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/home/freeman/anaconda3/envs/tf27p39/lib/python3.9/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/home/freeman/anaconda3/envs/tf27p39/lib/python3.9/site-pac

In [ ]:
E